# Google Drive ACL Sync Tool

This tool synchronizes permissions for a folder of Google Sheets based on a "master list" of emails.

**Features:**
* Reads a list of allowed emails from a Google Sheet.
* Scans a specific Drive folder for Spreadsheets.
* **Revokes** access for users not in the allowed list (Extra).
* **Grants** Editor access to users in the allowed list who are missing it.
* Generates a report of actions taken.

In [ ]:
!pip install --quiet google-api-python-client google-auth pandas

In [ ]:
from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
import pandas as pd

drive = build('drive', 'v3')
sheets = build('sheets', 'v4')

# --- CONFIGURATION ---
# Replace these with your actual IDs
FOLDER_ID = 'YOUR_TARGET_FOLDER_ID_HERE'
EMAIL_SHEET_ID = 'YOUR_MASTER_SHEET_ID_HERE'

In [ ]:
# 1. Load Allowed Emails
print("Loading allowed emails...")
result = sheets.spreadsheets().values().get(
    spreadsheetId=EMAIL_SHEET_ID,
    range='Sheet1!A:A'  # Assumes emails are in Column A
).execute()

expected_emails = {row[0].strip().lower() for row in result.get('values', []) if row}
print(f"Found {len(expected_emails)} allowed users.")

In [ ]:
# 2. Scan Target Folder
print("Scanning target folder...")
files = drive.files().list(
    q=f"'{FOLDER_ID}' in parents and mimeType='application/vnd.google-apps.spreadsheet' and trashed=false",
    fields="files(id, name)"
).execute().get('files', [])

print(f"Found tables: {len(files)}")

In [ ]:
# 3. Compare Permissions (Audit)
report = []

print("Auditing permissions...")
for file in files:
    perms = drive.permissions().list(
        fileId=file['id'],
        fields="permissions(emailAddress, role)"
    ).execute()

    actual_emails = {p['emailAddress'].lower() for p in perms.get('permissions', []) if 'emailAddress' in p}

    # Check for missing access
    for email in expected_emails:
        status = '✅ OK' if email in actual_emails else '❌ Missing'
        report.append({'File': file['name'], 'Email': email, 'Status': status})

    # Check for unauthorized access
    for email in actual_emails - expected_emails:
        report.append({'File': file['name'], 'Email': email, 'Status': '⚠️ Extra'})

print("Audit complete.")

In [ ]:
df = pd.DataFrame(report)
# Display preview
df.head()

In [ ]:
# 4. Revoke Unauthorized Access
from googleapiclient.errors import HttpError

print("Starting revocation process...")
for _, r in df.iterrows():
    if r.Status != '⚠️ Extra':
        continue

    file_id = None
    for f in files:
        if f['name'] == r.File:
            file_id = f['id']
            break
    if not file_id:
        continue

    # Get permission ID to delete
    perms = drive.permissions().list(fileId=file_id,
                                     fields='permissions(id,emailAddress)').execute()
    for p in perms.get('permissions', []):
        if p.get('emailAddress', '').lower() == r.Email.lower():
            try:
                drive.permissions().delete(fileId=file_id,
                                          permissionId=p['id']).execute()
                print(f"❌ Access revoked: {r.Email} -> {r.File}")
            except HttpError as e:
                print(f"Failed to revoke {r.Email} ({r.File}): {e}")

In [ ]:
# 5. Grant Missing Access
print("Starting access grant process...")
for _, r in df.iterrows():
    if r.Status != '❌ Missing':
        continue

    file_id = next((f['id'] for f in files if f['name'] == r.File), None)
    if not file_id:
        continue

    try:
        drive.permissions().create(
            fileId=file_id,
            body={
                'type': 'user',
                'role': 'writer',   # Editor role
                'emailAddress': r.Email
            },
            sendNotificationEmail=False
        ).execute()
        print(f'✅ Access granted: {r.Email} -> {r.File}')
    except HttpError as e:
        print(f'Failed to grant {r.Email} ({r.File}): {e}')

In [ ]:
# 6. Export Log
sheet_title = 'Access-Audit-Log-' + pd.Timestamp.now().strftime('%Y-%m-%d')
spreadsheet = sheets.spreadsheets().create(body={'properties': {'title': sheet_title}}).execute()
new_id = spreadsheet['spreadsheetId']

sheets.spreadsheets().values().update(
    spreadsheetId=new_id,
    range='A1',
    valueInputOption='RAW',
    body={'values': [df.columns.values.tolist()] + df.values.tolist()}
).execute()

print('Log saved. Link:')
print(f'https://docs.google.com/spreadsheets/d/{new_id}')